In [1]:
import sqlite3
import pandas as pd

def get_stock(code, connection, api):

        try:
            sql = "SELECT * FROM stocks WHERE code = '{}'".format(code)
            df = pd.read_sql(sql, connection, index_col=['code'])
        except:
            df = pd.DataFrame()
    
        if not df.empty:
            return df, True
    
        stock = api.Contracts.Stocks[code]
    
        stock_dict = {
            'code': [stock.code],
            'name': [stock.name],
            'category': [stock.category],
            'day_trade': [stock.day_trade.value]
        }
   
        df = pd.DataFrame(data=stock_dict)
        df = df.set_index('code')
        
        return df, False

In [2]:
import time
def update_stocks(daily_target, connection, api):
    
    main_df = pd.DataFrame()
    
    code_list = []
    
    for codes in daily_target.values():
        code_list.extend(codes)
        
    code_list = list(set(code_list))
    
    for code in code_list:
          df, in_db = get_stock(code, connection, api)
            
          if df is not None and not in_db:
                 main_df = main_df.append(df, sort=False)
                 time.sleep(1)  
                    
    if not main_df.empty: 
        main_df.to_sql('stocks', connection, if_exists='append')
        return main_df

In [3]:
daily_target = {
    pd.to_datetime('2021/6/15'): ['1905','2303','2409','2614','2618','3481','5608','6116']
}
daily_target

{Timestamp('2021-06-15 00:00:00'): ['1905',
  '2303',
  '2409',
  '2614',
  '2618',
  '3481',
  '5608',
  '6116']}

In [4]:
import shioaji as sj
api = sj.Shioaji()
api.login(
    person_id='N123569782',
    passwd='hons3146',
    contracts_cb=lambda security_type: print(f"{repr(security_type)} fetch done."))

connection = sqlite3.connect('data.db')

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


Exception: {'status': {'status_code': 400}, 'response': {'token': '', 'accounts': [], 'detail': 'ORA-20006:   passwd_err_3_times\nORA-06512: at "STSTX.SGTP_LOGIN_G", line 407\nORA-06512: at line 1', 'ip_address': '111.83.189.60', 'client_name': ''}}

In [ ]:
update_stocks(daily_target, connection, api)

In [5]:
get_stock('2303', connection, api)
get_stock('2303', connection, api)[0]
get_stock('2303', connection, api)[0].iloc[0]['day_trade']
get_stock('2303', connection, api)[0].iloc[0]['day_trade'] == 'Yes'

NameError: name 'connection' is not defined

In [6]:
codes = ['1905','2303','2409','2614','2618','3481','5608']
day_trading_codes = [code for code in codes if get_stock(code, connection, api)[0].iloc[0]['day_trade'] == 'Yes']
day_trading_codes

NameError: name 'connection' is not defined